In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


# Hola mundo
### Cargamos el resultado de los benchmark

In [15]:
col = ["test" + str(i) for i in range(1,5)]
scol = [str(i) for i in range(1,4)]

row =  ("seq",
        "omp",
        "cuda",
        "mpi")

srow = ("cycles",
        "real_time_nsec",
        "PAPI_TOT_INS",
        "PAPI_TOT_CYC")

mcol = pd.MultiIndex.from_product([col,scol])
mrow = pd.MultiIndex.from_product([row,srow])

data = np.loadtxt("results.csv", delimiter=',')

df = pd.DataFrame(data, index=mrow, columns=mcol)

test1                                     test2  \
                                1             2             3             1   
seq  cycles          1.468304e+10  1.439273e+10  1.691393e+10  1.336605e+11   
     real_time_nsec  4.591058e+09  4.500253e+09  5.288702e+09  4.179265e+10   
     PAPI_TOT_INS    2.602102e+10  2.602102e+10  2.602102e+10  2.080841e+11   
     PAPI_TOT_CYC    1.449790e+10  1.436644e+10  1.762524e+10  1.328230e+11   
omp  cycles          1.699579e+10  3.648458e+09  3.488470e+09  2.552303e+11   
     real_time_nsec  5.314186e+09  1.140785e+09  1.090775e+09  7.980462e+10   
     PAPI_TOT_INS    2.602102e+10  6.757684e+09  6.757642e+09  2.080841e+11   
     PAPI_TOT_CYC    1.699663e+10  3.376351e+09  3.451510e+09  2.526595e+11   
cuda cycles          3.693847e+09  3.854066e+09 -1.000000e+00  2.501805e+10   
     real_time_nsec  1.154978e+09  1.205087e+09 -1.000000e+00  7.822560e+09   
     PAPI_TOT_INS    6.498314e+09  6.848990e+09 -1.000000e+00  5.201880e+10   
     PAPI_TOT_CYC    3.360670e+09  3.389534e+09 -1.000000e+00  2.553759e+10   
mpi  cycles          3.805278e+09  3.785052e+09 -1.000000e+00  2.598758e+10   
     real_time_nsec  1.189865e+09  1.183508e+09 -1.000000e+00  8.126011e+09   
     PAPI_TOT_INS    2.620488e+09  2.738829e+09 -1.000000e+00  2.091548e+10   
     PAPI_TOT_CYC    1.602628e+09  1.630589e+09 -1.000000e+00  1.388399e+10   

                                                        test3                \
                                2             3             1             2   
seq  cycles          1.360803e+11  1.287597e+11  7.651385e+11  3.887410e+11   
     real_time_nsec  4.254900e+10  4.026098e+10  2.392419e+11  1.215499e+11   
     PAPI_TOT_INS    2.080841e+11  2.080841e+11  7.021894e+11  7.021892e+11   
     PAPI_TOT_CYC    1.338188e+11  1.340140e+11  7.608616e+11  3.974370e+11   
omp  cycles          3.311795e+10  3.429769e+10  2.551634e+11  1.058896e+11   
     real_time_nsec  1.035521e+10  1.072422e+10  7.978413e+10  3.310921e+10   
     PAPI_TOT_INS    5.402205e+10  5.402415e+10  2.080842e+11  1.823017e+11   
     PAPI_TOT_CYC    3.200694e+10  3.306304e+10  2.514709e+11  1.022004e+11   
cuda cycles          2.507042e+10 -1.000000e+00  8.379186e+10  8.382961e+10   
     real_time_nsec  7.839018e+09 -1.000000e+00  2.619976e+10  2.621185e+10   
     PAPI_TOT_INS    5.106091e+10 -1.000000e+00  1.752362e+11  1.713087e+11   
     PAPI_TOT_CYC    2.548680e+10 -1.000000e+00  8.661697e+10  8.672765e+10   
mpi  cycles          2.634810e+10 -1.000000e+00  9.465536e+10  9.483562e+10   
     real_time_nsec  8.238523e+09 -1.000000e+00  2.959759e+10  2.965320e+10   
     PAPI_TOT_INS    2.126851e+10 -1.000000e+00  7.294224e+10  7.325904e+10   
     PAPI_TOT_CYC    1.386705e+10 -1.000000e+00  4.378073e+10  4.469224e+10   

                                          test4                              
                                3             1             2             3  
seq  cycles          3.769964e+11 -1.000000e+00  1.973876e+12  1.228293e+12  
     real_time_nsec  1.178802e+11 -1.000000e+00  6.171845e+11  3.840642e+11  
     PAPI_TOT_INS    7.021892e+11 -1.000000e+00  1.664337e+12  1.664336e+12  
     PAPI_TOT_CYC    3.925234e+11 -1.000000e+00  2.055522e+12  1.277865e+12  
omp  cycles          1.005489e+11  2.483321e+12  4.921353e+11  3.643918e+11  
     real_time_nsec  3.143967e+10  7.764808e+11  1.538792e+11  1.139381e+11  
     PAPI_TOT_INS    1.823017e+11  1.664337e+12  4.320882e+11  4.320903e+11  
     PAPI_TOT_CYC    9.820710e+10  2.474615e+12  4.805681e+11  3.498608e+11  
cuda cycles         -1.000000e+00  1.989998e+11  1.978707e+11 -1.000000e+00  
     real_time_nsec -1.000000e+00  6.222263e+10  6.187023e+10 -1.000000e+00  
     PAPI_TOT_INS   -1.000000e+00  4.135083e+11  4.171440e+11 -1.000000e+00  
     PAPI_TOT_CYC   -1.000000e+00  2.061275e+11  2.057213e+11 -1.000000e+00  
mpi  cycles         -1.000000e+00  3.518702e+11  3.570217e+11 -1.000000e+00  
     real_ti